<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase4/blob/main/Fase4_Tech_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconhecimento Facial

Desafio numero 4: analisando rostos, sentimentos e atividades.

## Instalar Dependências

In [3]:
# Instalar dependências necessárias
!pip install google-generativeai opencv-python opencv-python-headless matplotlib gdown deepface mediapipe fpdf --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.6 MB/s eta 0:00:00


## Importar bibliotecas

In [4]:
# Importar bibliotecas
import gdown
import os
import cv2
import numpy as np
import google.generativeai as gemini
import time
import base64
from deepface import DeepFace
import dlib
import pandas as pd
import mediapipe as mp
from fpdf import FPDF

## Download do vídeo

In [5]:
# Baixar o vídeo diretamente do Google Drive
VIDEO_ID = "1B5PbZdUDi-r7Ac7BK3a3WdNppfQqM_Ne"  # ID do arquivo no Google Drive
VIDEO_PATH = "/content/video.mp4"
gdown.download(f"https://drive.google.com/uc?id={VIDEO_ID}", VIDEO_PATH, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1B5PbZdUDi-r7Ac7BK3a3WdNppfQqM_Ne
To: /content/video.mp4
100%|██████████| 38.3M/38.3M [00:00<00:00, 78.7MB/s]


'/content/video.mp4'

## Funções auxiliares

In [17]:
# Configuração do Mediapipe para detecção de atividades
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def analisar_emocao(frame):
    try:
        analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        return analysis[0]['dominant_emotion']
    except:
        return "desconhecido"

def detectar_atividade(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    return "Movimento detectado" if results.pose_landmarks else "Sem atividade"

def processar_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))
    detector = dlib.get_frontal_face_detector()
    dados = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        frame, num_faces = detectar_faces(frame, detector)
        emocao = analisar_emocao(frame)
        atividade = detectar_atividade(frame)
        dados.append([frame_count, num_faces, emocao, atividade])
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return dados

def gerar_relatorio(dados, relatorio_path):
    df = pd.DataFrame(dados, columns=['Frame', 'Rostos', 'Expressão', 'Atividade'])
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, "Relatório de Análise de Vídeo", ln=True, align='C')
    pdf.cell(200, 10, f"Total de Frames: {len(df)}", ln=True, align='L')
    pdf.cell(200, 10, f"Rostos Detectados: {df['Rostos'].sum()}", ln=True, align='L')
    pdf.output(relatorio_path)

## Fluxo principal

Execução das rotinas de processamento, usando as bibliotecas OpenCV, DLib, Mediapipe e Deepface. No final, gera um PDF.

In [20]:
def detectar_faces(frame, detector):
    # Convert to RGB and ensure it's a contiguous array with the correct dtype
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb_frame = np.ascontiguousarray(rgb_frame, dtype=np.uint8)

    # Detect faces using the RGB image
    faces = detector(rgb_frame)

    # Draw bounding boxes on detected faces (using the original frame)
    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return frame, len(faces)

# Processar vídeo com OpenCV, DLib, Mediapipe, Deepface
def main():
    video_path = "/content/video.mp4"
    output_path = "results/output_video.avi"
    relatorio_path = "results/relatorio.pdf"
    dados = processar_video(video_path, output_path)
    gerar_relatorio(dados, relatorio_path)
    print("Processamento concluído! Relatório gerado.")

# Executar o pipeline
main()

RuntimeError: Unsupported image type, must be 8bit gray or RGB image.